In [1]:
import lumicks.pylake as lk
import matplotlib.pyplot as plt
import numpy as np
d = lk.File("/home/kris/proj/.data/tweez/full.h5")

In [19]:
dir(d)
d

KeyError: "Unable to open object (object 'Distance 10' doesn't exist)"

In [16]:
channel = d.fdcurves
%matplotlib widget
zoomer = channel.range_selector

AttributeError: 'dict' object has no attribute 'range_selector'

In [3]:
zoomer.ranges

array([[1614933545777396200, 1614933562676429800]])

In [4]:
class fholder:
    def __init__(self,f):
        self.f = f

In [5]:
%matplotlib widget
channel = zoomer.slices[0]
print(zoomer.slices[0])
slicedict = {f'zoom{i}': fholder(d.downsampled_force2[slice(*zoomer.ranges[i])]) for i in range(len(zoomer.ranges))}
selector = lk.FdRangeSelector(slicedict)
plt.show()
#for channel in selector.slices:
 #   plt.figure()
 #   channel.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='curve', options=('zoom0',), value='zoom0'), Output()), _dom_classe…

In [6]:
selector.ranges

{'zoom0': array([[1614933547986420200, 1614933551741761000],
        [1614933551962663400, 1614933555497101800],
        [1614933556270260200, 1614933559252442600]])}

In [7]:
flat_ranges = []
for array in [selector.ranges[key] for key in selector.ranges]:
    for item in array:
        flat_ranges.append(item)
flat_ranges

[array([1614933547986420200, 1614933551741761000]),
 array([1614933551962663400, 1614933555497101800]),
 array([1614933556270260200, 1614933559252442600])]

In [8]:
fdc = lk.fdcurve.FDCurve(d, flat_ranges[0][0], flat_ranges[0][1], 'test', force = 2, distance = 1)
print(fdc.d.data)
d.fdcurves['test'] = fdc
plt.figure()
plt.plot(fdc.d.data, fdc.f.data)
plt.show()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
downsampled_forces = []
interp_distances = []
curves = {}
kernel_size = 50
target = 5000
for i, sel_range in enumerate(flat_ranges):
    slc = slice(*sel_range)
    forces = d.force1x[slc].data
    step = int(len(forces) / target)
    print(step)
    kernel_size = 5 * step
    timestamps = d.force1x[slc].timestamps
    kernel = np.ones(kernel_size) / kernel_size
    smoothed_forces = np.convolve(forces, kernel, mode='same')
    ds_forces = smoothed_forces[kernel_size:-kernel_size:step]
    #downsampled_forces.append(forces[::step])
    # this is ridiculous:
    #interp_distances.append(np.interp(timestamps, d.downsampled_force1[slc].timestamps,
                   #                  d.distance1[slc].data)[::step])
    smoothed_dists = np.convolve(d.distance1[slc].data, np.ones(3) / 3, mode='same')[1:]
    interp_dists = np.interp(timestamps[::step], d.downsampled_force1[slc].timestamps[1:],
                                smoothed_dists)

    
    identifier = f'split_curve{i}'
    curve = {'dist': interp_dists,
            'force': ds_forces - min(ds_forces)}  # bad maybe
    curves[identifier] = curve

56
58
56
55
58
62
67
63
63
67
69
65


In [13]:
plt.figure()
plt.plot(curves['split_curve0']['dist'])
plt.figure()
plt.plot(curves['split_curve0']['force'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
import csv
with open('/home/kris/proj/.data/tweez/split_curves.csv', 'w') as f:
    wr = csv.writer(f)
    for curve_id, curve in curves.items():
        wr.writerow([curve_id, *[dist for dist in curve['dist']]])
        wr.writerow([curve_id, *[force for force in curve['force']]])

In [12]:
[dist for dist in curves['split_curve0']['dist']]

[0.08438653291412557,
 0.08441803047580901,
 0.08444952803749244,
 0.08448102559917588,
 0.08451252316085932,
 0.08454402072254275,
 0.08457551828422619,
 0.08460701584590961,
 0.08463851340759306,
 0.0846700109692765,
 0.08470150853095992,
 0.08473300609264336,
 0.0847645036543268,
 0.08479600121601023,
 0.08482749877769367,
 0.0848589963393771,
 0.08489049390106054,
 0.08492199146274398,
 0.08495348902442741,
 0.08498498658611085,
 0.0850164841477943,
 0.08504798170947772,
 0.08507947927116116,
 0.08511097683284459,
 0.08514247439452803,
 0.08517397195621147,
 0.0852054695178949,
 0.08523696707957834,
 0.08526846464126178,
 0.08529996220294521,
 0.08533145976462865,
 0.08536295732631208,
 0.08539445488799552,
 0.08542595244967896,
 0.08545745001136239,
 0.08548894757304583,
 0.08552044513472927,
 0.0855519426964127,
 0.08558344025809614,
 0.08561493781977957,
 0.08564643538146301,
 0.08567793294314645,
 0.08570943050482988,
 0.08574092806651332,
 0.08577242562819676,
 0.0858039231898

In [14]:
with open('mycurves.csv', 'r') as f:
    lis = [line.split('\n')[0].split(',') for line in f]
    print(lis[0][0])
    print(np.asarray([float(x) for x in lis[0][1:]]))

split_curve0
[0.08438653 0.08441803 0.08444953 ... 0.08650409 0.08650409 0.08650409]


In [15]:
for dist, force in zip(*[iter(lis)]*2):
    print(dist[0], force[0])
    print(np.mean([float(x) for x in dist[1:]]), len(dist))
    print(np.mean([float(x) for x in force[1:]]), len(dist))

split_curve0 split_curve0
0.24224081516476356 5016
28.712270811726484 5016
split_curve1 split_curve1
0.24142194267491549 5016
29.050314181928652 5016
split_curve2 split_curve2
0.25560561072813703 5004
31.52449230806329 5004
split_curve3 split_curve3
0.251636002862274 5056
31.49011493999309 5056
split_curve4 split_curve4
0.24182986677016394 5035
30.193839367414093 5035
split_curve5 split_curve5
0.240067349962919 5064
28.617378455505428 5064
split_curve6 split_curve6
0.238385483425571 5016
28.886073363193702 5016
split_curve7 split_curve7
0.23950506776794445 5016
29.47167986157523 5016
split_curve8 split_curve8
0.23922994114203924 5016
29.34292611756373 5016
split_curve9 split_curve9
0.265074726166283 5069
33.04504931303689 5069
split_curve10 split_curve10
0.23949663709449256 5016
29.577881528248714 5016
split_curve11 split_curve11
0.23360012077196252 5064
30.0594093234766 5064
